<a href="https://colab.research.google.com/github/Edenshmuel/PapaJohns_Data_Science_Project/blob/main/Predicting_New_Categories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing libraries and reading data

In [1]:
!pip install fuzzywuzzy[speedup]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.3 MB/s eta 0:00:00


In [30]:
from google.colab import drive
import os
import shutil
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from fuzzywuzzy import process
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
cleaned_data = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/cleaned_data.csv')

In [33]:
category_mapping = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/category_mapping.csv')
desc_encoding_map = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/desc_encoding_map.csv')

In [34]:
cleaned_data

,Date,Year,Month,Day,Day_Name,Is_Weekend,Cleaned Description,Quantity,Category,Cleaned Description Normalized,...,Day_Name_cos,Month_sin,Month_cos,avg_quantity_all_time,std_quantity_all_time,num_days_sold,popularity_score,encoded_jewish_holiday,encoded_christian_holiday,encoded_portion_type
0,2022-01-01,2022,1,1,7,1,ביאנקה 14,1,Main Dish,ביאנקה 14,...,1.000000,5.000000e-01,0.866025,1.012442,0.124182,359,1.012442,9,7,1
1,2022-01-01,2022,1,1,7,1,תוספות,2,Topping,תוספות,...,1.000000,5.000000e-01,0.866025,1.968203,0.192883,235,1.968203,9,7,1
2,2022-01-01,2022,1,1,7,1,קוקה קולה 15 ליטר,1,Drink,קוקה קולה 15 ליטר,...,1.000000,5.000000e-01,0.866025,1.031121,0.260216,579,1.031121,9,7,1
3,2022-01-01,2022,1,1,7,1,קוקה קולה 15 ליטר,1,Drink,קוקה קולה 15 ליטר,...,1.000000,5.000000e-01,0.866025,1.031121,0.260216,579,1.031121,9,7,1
4,2022-01-01,2022,1,1,7,1,קוקה קולה 15 ליטר,1,Drink,קוקה קולה 15 ליטר,...,1.000000,5.000000e-01,0.866025,1.031121,0.260216,579,1.031121,9,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226041,2024-12-31,2024,12,31,3,0,קלאסית 16,1,Main Dish,קלאסית 16,...,-0.900969,-2.449294e-16,1.000000,1.205824,0.560581,633,1.205824,0,12,1
226042,2024-12-31,2024,12,31,3,0,קלאסית 16,1,Main Dish,קלאסית 16,...,-0.900969,-2.449294e-16,1.000000,1.205824,0.560581,633,1.205824,0,12,1
226043,2024-12-31,2024,12,31,3,0,קלאסית 16,1,Main Dish,קלאסית 16,...,-0.900969,-2.449294e-16,1.000000,1.205824,0.560581,633,1.205824,0,12,1
226044,2024-12-31,2024,12,31,3,0,קלאסית דקה 16,1,Main Dish,קלאסית דקה 16,...,-0.900969,-2.449294e-16,1.000000,1.149081,0.450818,542,1.149081,0,12,1


In [35]:
cleaned_data.isnull().sum()

,0
Date,0
Year,0
Month,0
Day,0
Day_Name,0
Is_Weekend,0
Cleaned Description,0
Quantity,0
Category,0
Cleaned Description Normalized,46


### Select data for model training
We use only the Cleaned Description Normalized column as input for training the category prediction model.
The Category column will be used as the target label for comparison.


In [36]:
model_data = cleaned_data[["Cleaned Description Normalized", "Category"]]

In [37]:
model_data = model_data.dropna(subset=["Cleaned Description Normalized", "Category"])

In [64]:
additional_examples = pd.DataFrame({
    'item_description': [
        # Desserts
        'עוגת שוקולד חמה', 'עוגת גבינה ניו יורק', 'קינוח שוקולד אישי', 'טירמיסו', 'בראוניז', 'עוגה', 'קינוח', 'קרם ברולה', 'אלפחורס',

        # Drinks
        'קולה דיאט', '1.5 מים מינרליים', 'ספרייט בקבוק', 'תפוחים', '1.5 פפסי', 'מים', 'אקסל', 'נביעות', 'זירו', "ליטר",

        # Sauces
        'שום שמיר', 'חריף', 'סלסה', 'ברביקיו מעושן', 'אלף האיים', 'רוטב', 'טבסקו',

        # Toppings
        'בולגרית', 'פטריות טריות', 'זיתים ירוקים', 'אננס', 'עגבניות מיובשות', 'גבינה', 'בצל', 'אנשובי',

        # Main courses
        'פיצה מרגריטה', 'פיצה טוסקנית 14', 'פיצה טבעונית אישית', 'פיצה גבינות מיוחדת', '8 פיצה נקניק חריף דקה', 'הטבעונית', 'המיוחדת 16', '14 כשרה לפסח', 'בוטנים דבש 16'

    ],
    'category': [
    # Desserts
    'Dessert', 'Dessert', 'Dessert', 'Dessert', 'Dessert', 'Dessert', 'Dessert', 'Dessert', 'Dessert',

    # Drinks
    'Drink', 'Drink', 'Drink', 'Drink', 'Drink', 'Drink', 'Drink', 'Drink', 'Drink' ,'Drink',

    # Sauces
    'Sauce', 'Sauce', 'Sauce', 'Sauce', 'Sauce', 'Sauce', 'Sauce',

    # Toppings
    'Topping', 'Topping', 'Topping', 'Topping', 'Topping', 'Topping', 'Topping', 'Topping',

    # Main Dish
    'Main Dish', 'Main Dish', 'Main Dish', 'Main Dish', 'Main Dish', 'Main Dish', 'Main Dish', 'Main Dish','Main Dish'
    ]
})

In [65]:
model_data = pd.concat([model_data, additional_examples], ignore_index=True)

In [66]:
print(model_data['Category'].value_counts(normalize=True))

Category
Main Dish    0.419009
Topping      0.300739
Drink        0.099248
Sauce        0.089690
Side Dish    0.047788
Dessert      0.043527
Name: proportion, dtype: float64


## Mapping categories from the file

In [67]:
# Category Mapping: Text to Code
category_to_index = dict(zip(category_mapping['Category'], category_mapping['Code']))
index_to_category = {v: k for k, v in category_to_index.items()}

## Balancing categories

In [69]:
samples_per_category = 150000
balanced_model_data = model_data.groupby('Category', group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), samples_per_category), random_state=42)
).reset_index(drop=True)

<ipython-input-69-a81f60436295>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_model_data = model_data.groupby('Category', group_keys=False).apply(


In [70]:
print(balanced_model_data['Category'].value_counts(normalize=True))

Category
Main Dish    0.419009
Topping      0.300739
Drink        0.099248
Sauce        0.089690
Side Dish    0.047788
Dessert      0.043527
Name: proportion, dtype: float64


## data fragmentation

In [71]:
X = balanced_model_data['Cleaned Description Normalized']

# Encoding the categories from names to numbers
y_encoded = balanced_model_data['Category'].map(category_to_index)

In [72]:
# Split into Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

## Model building and training

In [73]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    TfidfVectorizer(),
    XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
)

scores = cross_val_score(pipeline, X, y_encoded, cv=5, scoring='f1_macro')
print("F1 Macro (CV):", scores.mean())

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:59:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:59:41] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:59:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:00:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:00:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

F1 Macro (CV): 0.9993899960130477


In [74]:
model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))])

model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:00:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric='mlogloss',
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))])

## Predicting and returning category names

In [75]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=index_to_category.values()))

              precision    recall  f1-score   support

     Dessert       1.00      1.00      1.00      1968
       Drink       1.00      1.00      1.00      4486
   Main Dish       1.00      1.00      1.00     18939
       Sauce       1.00      1.00      1.00      4054
   Side Dish       0.99      1.00      1.00      2160
     Topping       1.00      1.00      1.00     13593

    accuracy                           1.00     45200
   macro avg       1.00      1.00      1.00     45200
weighted avg       1.00      1.00      1.00     45200



## New product prediction function (including security)

### 🧠 How the Model Handles Unknown or New Categories

This classification model is designed to predict the category of a product **based solely on its description** (`"Greek Salad"`, `"Papa Deal"`, `"Coca Cola"`).

#### 🟢 Standard Behavior:
- The model uses a trained `TF-IDF + XGBoost` pipeline to predict the **most likely category** from the known set (`'Main Dish'`, `'Drink'`, `'Dessert'`).
- These categories are based on the `category_mapping.csv` file and aligned with the internal system codes (1–6).

#### ⚠️ Special Handling for New/Unknown Products:
- If the model is **not confident enough** in its prediction (the top probability is **below a certain threshold**, such as 0.6),  
  it will **not return a specific category**.
- Instead, it returns a special label: **"⚠️ Category not recognized – Unclassified"**.

#### ✅ Why this is important:
- It ensures that **new or unusual products** (like limited-time offers or misspelled items) are not forced into incorrect categories.
- It also allows system operators to **review and manually classify** such items, or update the model over time.

> In summary: the model is capable of both confidently classifying known products and flagging new or unclear ones as "Unclassified".

In [76]:
def predict_from_input(model, index_to_category, additional_examples, threshold=0.6, fuzz_threshold=80):
    import re
    from fuzzywuzzy import process
    import numpy as np

    def clean_text(t):
        t = t.strip().lower()
        t = re.sub(r'^[הוי]', '', t)
        t = re.sub(r'\s+', ' ', t)
        return t

    keyword_map = {
        clean_text(row['item_description']): row['category']
        for _, row in additional_examples.iterrows()
    }

    print("Enter a product description (type 'exit' to quit):")
    while True:
        user_input = input("Description: ").strip()
        if user_input.lower() == 'exit':
            print("Exiting prediction mode.")
            break

        cleaned_input = clean_text(user_input)

        # Step 1: exact match
        if cleaned_input in keyword_map:
            print(f"[Keyword match] Category: {keyword_map[cleaned_input]}")
            continue

        # Step 2: fuzzy match
        best_match, score = process.extractOne(cleaned_input, list(keyword_map.keys()))
        if score >= fuzz_threshold:
            print(f"[Fuzzy match] Category: {keyword_map[best_match]} (match: {best_match}, score: {score})")
            continue

        # Step 3: model prediction
        probas = model.predict_proba([user_input])[0]
        max_proba = np.max(probas)
        predicted_index = np.argmax(probas)

        if max_proba < threshold:
            print("Category not recognized – Unclassified")
        else:
            print(f"[Model prediction] Category: {index_to_category[predicted_index]} (Confidence: {max_proba:.2f})")

## Example of a prediction:

In [77]:
predict_from_input(model, index_to_category, additional_examples, threshold=0.8)

Enter a product description (type 'exit' to quit):
Description: פאפיה דבש 14
[Model prediction] Category: Main Dish (Confidence: 1.00)
Description: פאי שוקולד
[Fuzzy match] Category: Dessert (match: עוגת שוקולד חמה, score: 86)
Description: קרם פיסטוק
[Model prediction] Category: Dessert (Confidence: 1.00)
Description: אצבעות שום
[Model prediction] Category: Side Dish (Confidence: 1.00)
Description: exit
Exiting prediction mode.


In [81]:
import joblib

joblib.dump(model, 'product_category_model.pkl')
joblib.dump(index_to_category, 'index_to_category.pkl')

['index_to_category.pkl']

In [79]:
from google.colab import files

files.download('product_category_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>